In [1]:
from langchain_community.chat_models import ChatOpenAI
import dotenv
import os
import joblib
import re

from discussion_agents.cog.functional.expel import categorize_experiences
from discussion_agents.cog.modules.memory.expel import ExpeLExperienceMemory

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)

In [2]:
experiences = joblib.load("../../../../tests/assets/expel/expel_experiences_10_fake.joblib")

In [3]:
categorize_experiences(experiences)

{'compare': [6, 7, 8, 9], 'success': [3, 5], 'fail': [0, 1, 2, 4]}

In [4]:
from discussion_agents.cog.prompts.react import REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES

fewshot_questions = re.findall(r'Question: (.+?)\n', REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES)
fewshot_keys = re.findall(r'Action \d+: Finish\[(.+?)\]', REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES)
blocks = re.split(r'(?=Question: )', REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES)[1:]  # Split and ignore the first empty result

fewshot_examples = []
for block in blocks:
    # Extract all thoughts, actions, and observations within each block
    thoughts = re.findall(r'(Thought \d+: .+?)\n', block)
    actions = re.findall(r'(Action \d+: .+?)\n', block)
    observations = re.findall(r'(Observation \d+: .+)', block)
    
    # Combine them into tuples and add to the examples list
    fewshot_examples.append(list(zip(thoughts, actions, observations)))

In [5]:
memory = ExpeLExperienceMemory(
    experiences, 
    success_traj_idxs=[3, 5],
    fewshot_questions=fewshot_questions,
    fewshot_keys=fewshot_keys,
    fewshot_examples=fewshot_examples
)

In [17]:
memory.experiences.keys()

dict_keys(['idxs', 'questions', 'keys', 'trajectories', 'reflections'])

In [8]:
memory.experiences['questions'][10:]

['What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?',
 'Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?',
 'Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?',
 'What profession does Nicholas Ray and Elia Kazan have in common?',
 "Which magazine was started first Arthur's Magazine or First for Women?",
 'Were Pavel Urysohn and Leonid Levin known for the same type of work?']

In [9]:
memory.experiences['keys'][10:]

['1,800 to 7,000 ft',
 'Richard Nixon',
 'The Saimaa Gesture',
 'director, screenwriter, actor',
 "Arthur's Magazine",
 'yes']

In [15]:
memory.experiences['trajectories'][10:]

[[(True,
   '1,800 to 7,000 ft',
   [('Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.',
     'Action 1: Search[Colorado orogeny]',
     'Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.'),
    ('Thought 2: It does not mention the eastern sector. So I need to look up eastern sector.',
     'Action 2: Lookup[eastern sector]',
     'Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.'),
    ('Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.',
     'Action 3: Search[High Plains]',
     'Observation 3: High Plains refers to one of two distinct land regions:'),
    ('Thought 4: I need to instead search High Plains (United States).',
     'Action 4:

In [18]:
memory.experiences['reflections'][10:]

[[], [], [], [], [], []]